In [50]:
import cv2
import tensorflow as tf
import numpy as np
from cv2 import dnn_superres
import os

def convert_to_binary_otsu(image):
    # Xác định ngưỡng tự động bằng phương pháp Otsu
    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return binary_image
def convert_to_binary_otsu(image):
    # Xác định ngưỡng tự động bằng phương pháp Otsu
    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary_image
def convert_to_binary_adaptive(image):
    # Áp dụng adaptive thresholding
    binary_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY, 11, 2)

    return binary_image

def process_up_scale_image(image):
    sr = dnn_superres.DnnSuperResImpl_create()
    path = "./EDSR_x4.pb"
    sr.readModel(path)

    sr.setModel("edsr", 3)

    result = sr.upsample(image)

    cv2.imwrite(".upscaled.png", result)
    # cv2.imwrite("./Cache/upscaled_thres.png", result_thres)
    return result
def convert(image):
    # Tạo một mảng mask dựa trên các điểm màu đen trong ảnh (0, 0, 0 là màu đen trong không gian màu RGB)
    mask_black = np.all(image == [0, 0, 0], axis=-1)

    # Tạo một mảng màu xanh lam (blue) có cùng kích thước như ảnh gốc
    blue_color = np.array([255, 0, 0], dtype=np.uint8)

    # Thay đổi màu của các điểm màu đen thành màu xanh lam (blue)
    image[mask_black] = blue_color
    return image
def rulerNumberClassification(image, conf):
    """
    Phân loại chữ số của một ảnh\n
    Args:
        image (Opencv) : Hình ảnh đầu vào
        conf : Ngưỡng
    Returns
        Một chữ số đại diện cho chữ số dự đoán từ ảnh đầu vào
    """
    # Đường dẫn tới model TensorFlow Lite (.tflite)
    model_path = 'best_model.tflite'

    # Load model TensorFlow Lite
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Lấy thông tin input và output tensors
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # cv2.imwrite("TestOrigin.png", image)
    # image = process_up_scale_image(image)
    # cv2.imwrite("TestUpscale.png", image)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # binary_image = convert_to_binary_otsu(image)
    # binary_image = cv2.bitwise_not(binary_image)
    # image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    # image = convert(image)
    cv2.imwrite("Test1212.png", image)
    # Resize ảnh về kích thước 28x28 như MNIST dataset

    # Đảm bảo ảnh có giá trị từ 0 đến 255
    # cv2.imwrite("TestBinary.png", binary_image)


    image = cv2.resize(image, (32, 32))  # Resize ảnh về kích thước 32x32
    image = np.expand_dims(image, axis=0)  # Mở rộng kích thước ảnh để tương thích với input shape của model
    image = image.astype(np.float32) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]

    # Đặt dữ liệu vào tensor input
    interpreter.set_tensor(input_details[0]['index'], image)

    # Chạy mô hình
    interpreter.invoke()

    # Lấy kết quả từ tensor output
    output_data = interpreter.get_tensor(output_details[0]['index'])
    #print(output_data)
    predicted_class = np.argmax(output_data)
    #print(predicted_class)
    cv2.putText(image, str(output_data[0][predicted_class]), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    # print("P : " + str(output_data[0][predicted_class]))
    if 0 <= predicted_class <= 8:
        if output_data[0][predicted_class] >= conf:
            return predicted_class + 1
    elif predicted_class == 9:
        if output_data[0][predicted_class] >= conf:
            return 0
    return None

# image = cv2.imread("./DataTest/numberDetectedCropped_4.png")
# rulerNumberClassification(image, 0.8)




In [62]:
labelCheck = 8
folder = "./Number Ruler Data/" + str(labelCheck)
predictLabels = []
for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Đảm bảo chỉ đọc các file ảnh có phần mở rộng là .jpg hoặc .png
            image = cv2.imread(file_path)
            predictLabels.append(rulerNumberClassification(image, 0.8))
# Tính toán độ chính xác
totalLabelPredict = 0
labelPredictTrue = 0
for label in predictLabels:
    totalLabelPredict += 1
    if label is not None:
        if label == labelCheck:
            labelPredictTrue += 1
print(totalLabelPredict)
print(labelPredictTrue)
print("Phần trăm dự đoán đúng label {}: {}".format(labelCheck, round(labelPredictTrue / totalLabelPredict, 2)))

210
107
Phần trăm dự đoán đúng label 8: 0.51
